## A Momentum Trading Strategy
#### Uses strength of price movements as signal for trade action
#### Focus on top and bottom quantiles of price moves (market herding behavior)
    Relies on:
    1) Volume
        high-interest, low-lack of interest
    2) Volatility
        larger price swings
    3) Time frame
        expected duration (day vs position trading)
#### Will assess strength of momentum across multiple assets at the same time in a cross-sectional manner
    - long leg: highly ranked assets with an expectation of appreciation
    - short leg: poorly ranked assets showing signs of decline

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import numpy as np
import yfinance as yf

/Users/elibemiss/anaconda3/lib/python3.10/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
# Using DJIA
def fetch_info():
    try: 
        url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
        headers = {'Accept': 'application/json, text/plain, */*',
                   'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}
        #send GET request
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        # get symbols table
        tables = soup.find_all('table')
        #convert to df and clean
        df = pd.read_html(str(tables))[1]
        df.drop(columns = ['Notes'], inplace=True)
        return df
    except:
        print('Error loading data')
        return None

In [5]:
# get DJIA stocks
dj_df = fetch_info()
dj_df.head()

,Company,Exchange,Symbol,Industry,Date added,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,1.54%
1,American Express,NYSE,AXP,Financial services,1982-08-30,3.64%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,4.80%
3,Amazon,NASDAQ,AMZN,Retailing,2024-02-26,2.93%
4,Apple,NASDAQ,AAPL,Information technology,2015-03-19,3.04%


In [7]:
# extract tickers to list
tickers = dj_df.Symbol.values.tolist()

In [29]:
# Download stock prices
df = yf.download(tickers, start = '2021-01-01', end = '2022-09-01')
df.head()

[*********************100%%**********************]  30 of 30 completed


Price        Adj Close                                                  \
Ticker            AAPL        AMGN        AMZN         AXP          BA   
Date                                                                     
2021-01-04  126.830055  202.947876  159.331497  112.457108  202.720001   
2021-01-05  128.398178  203.932800  160.925507  113.057312  211.630005   
2021-01-06  124.076096  208.848450  156.919006  117.239670  211.029999   
2021-01-07  128.309982  209.537918  158.108002  116.312317  212.710007   
2021-01-08  129.417435  213.540268  159.134995  116.427040  209.899994   

Price                                                                 ...  \
Ticker             CAT         CRM       CSCO        CVX         DIS  ...   
Date                                                                  ...   
2021-01-04  169.422394  219.680634  39.525215  73.209129  176.289490  ...   
2021-01-05  170.808304  220.887177  39.543194  75.188232  177.043564  ...   
2021-01-06  180.314209  215.532516  39.920830  77.608093  177.718216  ...   
2021-01-07  180.658310  217.357300  40.424339  77.971069  177.182449  ...   
2021-01-08  180.686249  221.405685  40.514248  78.714310  177.291595  ...   

Price        Volume                                                           \
Ticker          MMM       MRK      MSFT      NKE        PG      TRV      UNH   
Date                                                                           
2021-01-04  3583455  10502637  37130100  6178500   8330900  1559700  4203800   
2021-01-05  2745179  10956526  23823000  4163100   6856400   937700  3160500   
2021-01-06  4002294  10521396  35930700  6072900  10578000  1278900  6561400   
2021-01-07  7012626  10447931  27694500  5256900   7355400  1258100  3366700   
2021-01-08  5750488   9563105  22956200  3950500   7448500   995100  2947800   

Price                                     
Ticker             V        VZ       WMT  
Date                                      
2021-01-04  10318300  19129800  32182200  
2021-01-05   6869700  16220000  26498100  
2021-01-06   7206200  22678500  21918900  
2021-01-07  10967900  19079900  20538000  
2021-01-08   6513000  28411600  24478200  

[5 rows x 180 columns]

In [32]:
prices = df['Adj Close']

In [33]:
# chain breakdown:
    # first takes absolute price and calculates percentage change daily
    # resample groups these daily change data by month
    # the lambda x function is performed on each group 
        # x+1 : ex. 0.05 --> 1.05...
        # .prod() : multiplies all group elements to get cumulative return
        # -1 : subtract 1 to get terminal return in pct terms
monthly_returns = prices.pct_change().resample('M').agg(lambda x: (x+1).prod()-1)

In [34]:
monthly_returns.head()

Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-31,0.019705,0.065164,0.006141,-0.011626,-0.042078,0.009129,0.023830,0.014104,0.005785,-0.053523,...,0.022052,-0.048049,0.065552,-0.046467,-0.064117,0.004792,-0.045551,-0.112555,-0.059709,-0.041220
2021-02-28,-0.079712,-0.061457,-0.035328,0.163427,0.091766,0.180704,-0.040167,0.006505,0.190300,0.124101,...,0.004796,-0.057740,0.004118,0.010959,-0.036503,0.067498,-0.004077,0.100749,0.010046,-0.075237
2021-03-31,0.007340,0.106216,0.000372,0.048833,0.201453,0.074069,-0.021386,0.152440,0.047900,-0.023910,...,0.100651,0.070880,0.014588,-0.014023,0.096333,0.039349,0.123929,-0.003108,0.051537,0.049855
2021-04-30,0.076218,-0.036855,0.120663,0.084205,-0.080127,-0.011769,0.087082,-0.008412,-0.016414,0.008129,...,0.023147,-0.033597,0.069602,-0.002032,-0.008539,0.028324,0.071841,0.103103,0.004501,0.030037
2021-05-31,-0.050497,-0.000113,-0.070470,0.044213,0.054244,0.056859,0.033779,0.039088,0.019312,-0.039619,...,0.037507,0.018658,-0.007627,0.031031,0.010718,0.032588,0.032899,-0.025389,-0.022495,0.019139


In [35]:
# get historical cumulative returns of past 6 mo as terminal return of current monrth
past_cum_return = (monthly_returns + 1).rolling(6).apply(np.prod) - 1
past_cum_return.tail()

Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-30,0.055282,0.145886,-0.262956,0.010736,-0.281070,0.041989,-0.412927,-0.113703,0.398765,-0.339741,...,-0.178424,0.025870,-0.159874,-0.251544,0.135201,0.075016,0.111296,0.009834,-0.104956,0.031991
2022-05-31,-0.097105,0.312459,-0.314473,0.114407,-0.335861,0.127166,-0.437675,-0.168020,0.576555,-0.237819,...,-0.104585,0.251228,-0.174002,-0.294875,0.034117,0.231826,0.125287,0.098834,0.045157,-0.074745
2022-06-30,-0.227937,0.099514,-0.362932,-0.144964,-0.320882,-0.126977,-0.350569,-0.318526,0.256954,-0.390535,...,-0.256987,0.209991,-0.232991,-0.383890,-0.111276,0.092639,0.029568,-0.088243,0.000517,-0.153405
2022-07-31,-0.067650,0.107674,-0.097768,-0.137831,-0.204405,-0.004821,-0.208958,-0.172052,0.270547,-0.257886,...,-0.120069,0.115272,-0.093268,-0.220182,-0.123739,-0.034896,0.155141,-0.058846,-0.110640,-0.048404
2022-08-31,-0.045120,0.077805,-0.174476,-0.213520,-0.219587,-0.003686,-0.258443,-0.185377,0.116724,-0.245049,...,-0.146346,0.133770,-0.121004,-0.216718,-0.104404,-0.049338,0.098460,-0.077169,-0.201574,-0.007636


In [36]:
# create measurement and formation periods
import datetime as dt
end_of_measurement = dt.datetime(2022,6,30)
formation = dt.datetime(2022,7,31)

In [37]:
# the following six month terminal returns represent the relative momentum
end_of_measurement_returns = past_cum_return.loc[end_of_measurement]
end_of_measurement_returns = end_of_measurement_returns.reset_index()
end_of_measurement_returns.head()

,Ticker,2022-06-30 00:00:00
0,AAPL,-0.227937
1,AMGN,0.099514
2,AMZN,-0.362932
3,AXP,-0.144964
4,BA,-0.320882


In [40]:
# highest and lowest momentum: 
end_of_measurement_returns.loc[end_of_measurement_returns.iloc[:,1].idxmax()]
end_of_measurement_returns.loc[end_of_measurement_returns.iloc[:,1].idxmin()]

Ticker                      DIS
2022-06-30 00:00:00   -0.390535
Name: 9, dtype: object

In [43]:
# create quantiles to group stocks into top and bottom
end_of_measurement_returns['rank'] = pd.qcut(end_of_measurement_returns.iloc[:,1], 5, labels = False)
end_of_measurement_returns.head()

,Ticker,2022-06-30 00:00:00,rank
0,AAPL,-0.227937,1
1,AMGN,0.099514,4
2,AMZN,-0.362932,0
3,AXP,-0.144964,2
4,BA,-0.320882,0


In [51]:
# long and short
long_stocks = end_of_measurement_returns.loc[end_of_measurement_returns['rank']==4, "Ticker"].values
short_stocks = end_of_measurement_returns.loc[end_of_measurement_returns['rank']==0, "Ticker"].values
print(f'Long stocks: {long_stocks}')
print(f'Short stocks: {short_stocks}')

Long stocks: ['AMGN' 'CVX' 'IBM' 'KO' 'MRK' 'TRV']
Short stocks: ['AMZN' 'BA' 'CRM' 'DIS' 'HD' 'NKE']


In [54]:
from dateutil.relativedelta import relativedelta
long_return_df = monthly_returns.loc[formation + relativedelta(months=1), monthly_returns.columns.isin(long_stocks)]
long_return_df

Ticker
AMGN   -0.021474
CVX    -0.026156
IBM    -0.005517
KO     -0.038336
MRK    -0.044549
TRV     0.018526
Name: 2022-08-31 00:00:00, dtype: float64

In [55]:
short_return_df = monthly_returns.loc[formation + relativedelta(months=1), monthly_returns.columns.isin(short_stocks)]
short_return_df

Ticker
AMZN   -0.060615
BA      0.005900
CRM    -0.151614
DIS     0.056362
HD     -0.035350
NKE    -0.073704
Name: 2022-08-31 00:00:00, dtype: float64

In [56]:
momentum_profit = long_return_df.mean() - short_return_df.mean()
momentum_profit

0.023585567717981072

In [58]:
df_dji = yf.download("^DJI", start = '2021-01-01', end = '2022-09-01')
df_dji.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,30627.470703,30674.279297,29881.820312,30223.890625,30223.890625,475080000
2021-01-05,30204.250000,30504.890625,30141.779297,30391.599609,30391.599609,350910000
2021-01-06,30362.779297,31022.650391,30313.070312,30829.400391,30829.400391,500430000
2021-01-07,30901.179688,31193.400391,30897.859375,31041.130859,31041.130859,427810000
2021-01-08,31069.580078,31140.669922,30793.269531,31097.970703,31097.970703,381150000


In [60]:
buy_n_hold = df_dji['Adj Close'].pct_change().resample('M').agg(lambda x: (x+1).prod()-1)
buy_n_hold.head()

Date
2021-01-31   -0.007983
2021-02-28    0.031677
2021-03-31    0.066247
2021-04-30    0.027085
2021-05-31    0.019324
Freq: M, Name: Adj Close, dtype: float64

In [61]:
buy_n_hold.loc[formation + relativedelta(months=1)]

-0.04063613884907047